<a href="https://colab.research.google.com/github/puraminy/mini_proj2/blob/master/Hopfiled_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:


import numpy as np
import matplotlib.pyplot as plt


data = {
    "1": [1, 1, 0, 0, 0, 1, 1, 1,
          1, 1, 0, 0, 0, 1, 1, 1,
          1, 1, 1, 0, 0, 1, 1, 1,
          1, 1, 1, 0, 0, 1, 1, 1,
          1, 1, 1, 0, 0, 1, 1, 1,
          1, 1, 1, 0, 0, 1, 1, 1,
          1, 1, 1, 0, 0, 1, 1, 1,
          1, 1, 1, 0, 0, 1, 1, 1],

    "2": [0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          1, 1, 1, 1, 1, 1, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 1, 1, 1, 1, 1, 1,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0],
    
    
    "3": [0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          1, 1, 1, 1, 1, 1, 0, 0,
          1, 1, 0, 0, 0, 0, 0, 0,
          1, 1, 0, 0, 0, 0, 0, 0,
          1, 1, 1, 1, 1, 1, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0],

    "4": [1, 0, 0, 1, 1, 0, 0, 1,
          1, 0, 0, 1, 1, 0, 0, 1,
          1, 0, 0, 1, 1, 0, 0, 1,
          1, 0, 0, 0, 0, 0, 0, 1,
          1, 0, 0, 0, 0, 0, 0, 1,
          1, 1, 1, 1, 1, 0, 0, 1,
          1, 1, 1, 1, 1, 0, 0, 1,
          1, 1, 1, 1, 1, 0, 0, 1],

    "5": [0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 1, 1, 1, 1, 1, 1,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          1, 1, 1, 1, 1, 1, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0],

    "6": [0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 1, 1, 1, 1, 1, 1,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 1, 1, 1, 1, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0],

   "7": [1, 0, 0, 0, 0, 0, 0, 0,
          1, 0, 0, 0, 0, 0, 0, 0,
          1, 1, 1, 1, 1, 1, 0, 0,
          1, 1, 1, 1, 0, 0, 0, 1,
          1, 1, 1, 0, 0, 0, 0, 1,
          1, 1, 1, 0, 0, 1, 1, 1,
          1, 1, 1, 0, 0, 1, 1, 1,
          1, 1, 1, 0, 0, 1, 1, 1], 

    "8": [1, 1, 1, 1, 1, 1, 1, 1,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 1, 1, 1, 1, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 1, 1, 1, 1, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          1, 1, 1, 1, 1, 1, 1, 1],

    "9": [1, 0, 0, 0, 0, 0, 0, 0,
          1, 0, 0, 0, 0, 0, 0, 0,
          1, 0, 0, 1, 1, 1, 0, 0,
          1, 0, 0, 0, 0, 0, 0, 0,
          1, 0, 0, 0, 0, 0, 0, 0,
          1, 1, 1, 1, 1, 1, 1, 0,
          1, 1, 0, 0, 0, 0, 0, 0,
          1, 1, 0, 0, 0, 0, 0, 0],

    "0": [0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 1, 1, 1, 1, 0, 0,
          0, 0, 1, 1, 1, 1, 0, 0,
          0, 0, 1, 1, 1, 1, 0, 0,
          0, 0, 1, 1, 1, 1, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0],
}

# convert data to biploar 
for key,value in data.items():
    data[key]=[2*x-1 for x in value]
    
n_dim = 8

# Number of neurons
n_neurons = n_dim * n_dim

#Generate test data by adding noise to train data
def gen_test_data(n_test=200, distort=0.0):
  test_data = []
  for d in range(n_test):
      r_i = np.random.randint(0, n_train)
      base_pattern = np.array(train_data[r_i])
      noise = 1 * (np.random.random(base_pattern.shape) > distort)
      np.place(noise, noise == 0, -1)
      noisy_pattern = np.multiply(base_pattern, noise)
      test_data.append((base_pattern, noisy_pattern))
   
  return test_data

#Training 

def train(number_of_neurons, training_data):
    patterns_num = len(training_data)
    w = np.zeros([number_of_neurons, number_of_neurons])
    mean = np.sum([np.sum(t) for t in training_data]) / (patterns_num * number_of_neurons)
    for i in range(patterns_num):
        data = training_data[i] - mean
        w += np.outer(data, data)
    for diag in range(number_of_neurons):
        w[diag][diag] = 0
    return w

#Calculate the Hamming distance between two data
def ham_dist(s1, s2):
    assert len(s1) == len(s2)
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))  


# Reconstructing a pattern

def reconstruct_pattern(weights, data, bias=0):
  res = np.array(data)
# #     print("RES:",res)
  for _ in range(5):
    a = weights.dot(res) - bias
    res = np.sign(a)
  return res

#   for _ in range(steps):
#       for i in range(len(res)):
#           raw_v = np.dot(weights[i], res)  
#           if raw_v > 0: 
#               res[i] = 1
# #           elif raw_v == 0:
# #               res[i] = 0
#           else:
#               res[i] = -1
#   return res
  
  
# Testing
def test(weights, testing_data, bias =0):
    success = 0.0

    output_data = []
    hd = []
    for data in testing_data:
        true_data = data[0]
        noisy_data = data[1]
        predicted_data = reconstruct_pattern(weights, noisy_data, bias)
        if np.array_equal(true_data, predicted_data):
            success += 1.0

        output_data.append([true_data, noisy_data, predicted_data])

    return (success / len(testing_data)), output_data
  

def ham_dist_patterns(train_data, n_train):
  n = len(train_data)
  total_dist = 0
  i = 0
  mat = np.zeros((n_train, n_train))
  num=0
  for p,q in train_data.items():
    if i > n_train: break
    j = 0
    for s,r in train_data.items():
        if j> i and j < n_train:
          dist = ham_dist(q,r)
          mat[i][j]=dist
#           print(f"Hamming distance between {p} and {s} is {dist}");
          total_dist+= dist
          num+=1
        j+=1
    i+=1
  return total_dist, total_dist/num, mat

# Train 

data = dict((k, data[k]) for k in ('1','2','3','4','5','6','7','8','9','0'))

n_train = len(data)

train_data = [np.array(d) for d in data.values()][:n_train]

hd_total, hd_avg, hd_mat = ham_dist_patterns(data, n_train)

teams_list = [x for x in data.keys()]
teams_list = teams_list[:n_train]
row_format ="{:>10}" * (len(teams_list) + 1)
print(row_format.format("", *teams_list))
for team, row in zip(teams_list, hd_mat):
    print(row_format.format(team, *row))
    
print("\n\nTotal distance:",hd_total)
print("\n\nAverage distance:",hd_avg)

W = train(n_neurons, train_data)

# Test
distorts = [0.0, 0.1, 0.3]
for d in distorts:
  print("distort:",d )
  test_data = gen_test_data(n_test=1000,distort=d)
  accuracy, op_imgs = test(W, test_data)
  print("Accuracy of the network is %f" % (accuracy * 100))

print("\nTesting the effect of bias")

for b in range(10):
  print("distort: 0.3; bias:",b )
  test_data = gen_test_data(n_test=1000,distort=0.3)
  accuracy, op_imgs = test(W, test_data, bias=b)
  print("Accuracy of the network is %f" % (accuracy * 100))

"""#### Plot test result"""

def plot_images(images, title, no_i_x, no_i_y=3):
    fig = plt.figure(figsize=(10, 15))
    fig.canvas.set_window_title(title)
    images = np.array(images).reshape(-1, 8, 8)
    for i in range(no_i_x):
        for j in range(no_i_y):
            ax = fig.add_subplot(no_i_x, no_i_y, no_i_x * j + (i + 1))
            ax.matshow(images[no_i_x * j + i], cmap="gray")
            plt.xticks(np.array([]))
            plt.yticks(np.array([]))

            if j == 0 and i == 0:
                ax.set_title("Real")
            elif j == 0 and i == 1:
                ax.set_title("Distorted")
            elif j == 0 and i == 2:
                ax.set_title("Reconstructed")
                
# plot_images(op_imgs, "Reconstructed Data", 20)
# plt.show()

                

                   1         2         3         4         5         6         7         8         9         0
         1       0.0      42.0      38.0      36.0      42.0      44.0      17.0      40.0      35.0      46.0
         2       0.0       0.0       8.0      32.0       8.0       6.0      27.0      20.0      13.0      12.0
         3       0.0       0.0       0.0      30.0       8.0      10.0      23.0      24.0      11.0      16.0
         4       0.0       0.0       0.0       0.0      30.0      32.0      27.0      28.0      21.0      40.0
         5       0.0       0.0       0.0       0.0       0.0       2.0      31.0      20.0      13.0      12.0
         6       0.0       0.0       0.0       0.0       0.0       0.0      33.0      18.0      15.0      10.0
         7       0.0       0.0       0.0       0.0       0.0       0.0       0.0      33.0      20.0      33.0
         8       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0      23.0      24.0
 